In [1]:
import re
import os
import xml.etree.ElementTree as ET
import mysql.connector 
import pandas as pd

In [2]:
class Datos_xml:

# Esta clase almacenará la función para leer y limpiar los datos xml.
# Como parámetro recibirá el nombre del archivo.

    def __init__(self, nombre_archivo):
        self.nombre_archivo = nombre_archivo

    def lectura_xml(self): 
        import re
        import os
        import xml.etree.ElementTree as ET

        tree = ET.parse(self.nombre_archivo)
        root = tree.getroot()

        self.xml_lista = []
        for child in root:
            xml_dict = {}
            for subchild in child:
                xml_dict[subchild.tag] = subchild.text
            self.xml_lista.append(xml_dict) # Es una lista que contiene diccionarios
        return self.xml_lista

    def actualización_género(self,xml_lista):
    # Esta función actualiza los datos de la columna género. 
    # Como argumento recibe la variable creada con el método lectura_xml.

        self.xml_lista = xml_lista
        
        for i in xml_lista:
            
            if i["gender"] == '0':
                i["gender"] = "Man"
            elif i["gender"] == '1':
                i["gender"] = "Woman"
            elif i["gender"] == '2':
                i["gender"] = "Nonbinary"
            elif i["gender"] == '3':
                i["gender"] = "Prefer not to say"
            elif i["gender"] == '4':
                i["gender"] = "Prefer to self-describe"

        return xml_lista

    def limpieza_datos(self,xml_lista):

    # Con esta función se preparan los datos para su inserción:
    # Como argumento recibe la variable creada con el método atualización genero.

        self.xml_lista = xml_lista

        # Eliminar la primera columna
        try:
            for i in xml_lista:
                i.pop("level_0")
                i["index"] = int(i["index"])
        except: pass

        # Convertir los datos de la columna index en tipo integer

        try:
            for i in xml_lista:
                i["index"] = int(i["index"])
        except: pass

        # Aplicar la condición if para evitar problemas con index_sql, se aplica el método setdefault para buscar la key index_sql y asignarle un value de tipo integer.

        try:
            for i in xml_lista:
                if (xml_lista[0] != "index_sql"):
                    i.setdefault("index_sql", int(i["index"]))
        except: pass

        # Convertir los datos en una lista de tuplas.
    
        tupla_xml = []

        for i in xml_lista:
            tupla_xml.append(tuple(i.values()))
    
        return tupla_xml


In [3]:
archivo_xml = Datos_xml ("data_xml.xml")

In [4]:
lectura_xml = archivo_xml.lectura_xml()

In [5]:
genero_xml = archivo_xml.actualización_género(lectura_xml)

In [6]:
archivo_xml_limpio = archivo_xml.limpieza_datos(genero_xml)

In [7]:
print(archivo_xml_limpio[0])

(1, '784', '50-54', 'Man', 1)


In [8]:
query_xml = "INSERT INTO data_xml(index_xml, `time`, age, gender, index_sql) VALUES (%s,%s,%s,%s,%s);"

In [8]:
bbdd = "proyecto_1_equipo_1"

In [12]:
def insercion_datos(query,fichero_limpio):

    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                host='127.0.0.1', database= 'proyecto_1_equipo_1')

    cursor = cnx.cursor()
    
    try: 
        
        cursor.execute("SET foreign_key_checks = 0;")
        cursor.executemany(query, fichero_limpio)
        cnx.commit()
        print(cursor.rowcount, "registros insertados correctamente.")
        # mycursor.execute("SET foreign_key_checks = 1;")

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    
    
    cursor.close()
    cnx.close()

In [17]:
insercion_datos(query_xml,archivo_xml_limpio,bbdd)

25972 registros insertados correctamente.


### CONSULTAS

- Tablas que componen una base de datos

In [18]:
mydb = mysql.connector.connect(
              user='root', password='AlumnaAdalab',
              host='127.0.0.1',
              database='proyecto_1_equipo_1'
)

mycursor = mydb.cursor()

try: 
    mycursor.execute("SHOW TABLES")

    for x in mycursor:
      print(x)

except mysql.connector.Error as err:
    print(err)


('tabla_sql',)
('tabla_txt',)
('tabla_xml',)


- Tabla_xml

In [19]:
# importamos pandas
import pandas as pd

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='proyecto_1_equipo_1')

mycursor = cnx.cursor()

mycursor.execute("SELECT * FROM data_xml")

myresult = mycursor.fetchall()

df = pd.DataFrame(myresult, columns = ["index_xml","time","age","gender","index_sql"])

cnx.close()

In [20]:
# mostramos las 10 primeras filas del dataframe usando el método .head()
df.head(10)

,index_xml,time,age,gender,index_sql
0,1,784,50-54,Man,1
1,2,924,22-24,Man,2
2,3,575,45-49,Man,3
3,4,781,45-49,Man,4
4,5,1020,25-29,Woman,5
5,6,141,18-21,Woman,6
6,7,484,30-34,Man,7
7,8,1744,22-24,Man,8
8,9,655,30-34,Man,9
9,10,1777,40-44,Man,10


- Tabla_sql

In [21]:
# importamos pandas
import pandas as pd

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='proyecto_1_equipo_1')

mycursor = cnx.cursor()

mycursor.execute("SELECT * FROM data_sql")

myresult = mycursor.fetchall()

df = pd.DataFrame(myresult, columns = ["index_sql","q10_part_1","q10_part_2","q10_part_3", "q10_part_4", "q10_part_5","q10_part_6","q10_part_7", "q10_part_8","q10_part_9","q10_part_10","q10_part_11","q10_part_12","q10_part_13","q10_part_14","q10_part_15","q10_part_16","q10_other","d482xta"])

#Cerramos la conexion
cnx.close()

In [22]:
# mostramos las 10 primeras filas del dataframe usando el método .head()
df.head()

,index_sql,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,d482xta
0,1,Kaggle Notebooks,Colab Notebooks,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,2,Kaggle Notebooks,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,3,NULL,Colab Notebooks,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Google Cloud Datalab,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,5,NULL,Colab Notebooks,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
